In [1]:
import requests
import pandas as pd
import json
import geopandas
import operator
from prettytable import PrettyTable
import time
import numpy as np
from bs4 import BeautifulSoup

In [2]:
url = 'https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv'
covid_county = pd.read_csv(url)

We are pulling in data from usafactsstatic.com, that gives us information on how many covid cases each county had per day starting in late February. This data is updated daily but for the purpouses of this project, we collected data until May, 8th. Some of the issues with this dataset could be that for each state there are some cases that are unallocated to a county.

In [3]:
df = covid_county# rename dataframe to df.


In [4]:
covid_county = df.drop(df[(df['County Name'] == 'Statewide Unallocated')].index)# drop unallocated data

In [5]:
covid_county['total_covid']  = df['5/8/20']

In [6]:
df = pd.DataFrame(covid_county,columns = ['County Name','State','total_covid','countyFIPS'])
#segment the data into a dataframe

We segmented the data and only kept these the columns with countyname, state, all the covid cases in each county until May 8th and the county fips. This is a unique federal id given to each county.

In [7]:
zips = pd.read_csv('../../data/uszips.csv')# import zip code data

This dataframe has zip codes for every county in the U.S. That we can merge the hospital data with the zip codes and get the proper county for each.

In [8]:
hospitals = pd.read_csv('../../data/Hospitals.csv')
hospitals.columns = hospitals.columns.str.replace('ZIP','zip')# replace the name of the zip column

This dataframe, has every hospital in the U.S by county. We can add them to our dataframe and get a better sense of health resources for each county in the U.S.A This will help ful in understanding how counties can deal with a covid outbreak and a natural disastor at the same time.

In [9]:
med = pd.merge(hospitals,zips,on='zip')
med.columns = med.columns.str.replace('county_name','County')

We merged these dataframes on the zip code  column and created one dataframe that has both zip code and county. 

In [10]:
med['County'] = [i + " " + "County" for i in med['County']]# add the word county to the county list


In [11]:
parishes = med.loc[med['STATE'] == 'LA']
parishes['County'] = parishes['County'] + ' Parish'# for counties in Lousiana we need we need to add parish instead of County
parishes['STATE'] = 'LA'
(med.loc[med['STATE'] == 'LA']) = parishes



/Users/aryehgelfand/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/aryehgelfand/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


We need to add the word parish to the counties in Louisiana.

In [12]:
med_beds = med.groupby('County')['BEDS'].sum()# add up all the hospital beds in each county
med_hospitals = (med.groupby('County')['NAME']).value_counts().groupby('County').sum()#add up all the # hospitals in each county 


We created new columns with the number of beds and hospitals in each county, that way we can use this data in our predictive model. This will help us measure the potential impact of covid-19 and a simulatanous natural disastor on health resources.

In [13]:
counties = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')

We imported covid data by county including cases and deaths per county for every county in the U.S

In [14]:
counties.drop(columns=['date'],inplace=True)#drop the date column, this the date the data was gathered

In [15]:
counties.columns = counties.columns.str.replace('county','County Name')# replace county name to conform to our hospital dataframe

In [16]:
grouped_counties_cases = counties.groupby('County Name')['cases'].sum()# add up all cases for each county
grouped_counties_deaths = counties.groupby('County Name')['deaths'].sum()# add all deaths for every county

We added up all deaths for every county and cases for county by grouping our dataframe by county and then adding up all deaths and cases for each

In [17]:
county_cases = pd.merge(grouped_counties_cases,grouped_counties_deaths,on='County Name')# merge county and cases on County name
df = pd.merge(df,county_cases,left_on='County Name',right_on = 'County Name',how='left')# merge the original dataframe with the combined cases and deaths dataframe

We added the county cases and deaths to the dataframe that had hospital data.

In [18]:
df.fillna(0, inplace=True)
df.columns = df.columns.str.replace('County Name','County')# conformed the county column to the original

In [19]:
df = pd.merge(df,med_beds,left_on='County',right_on = 'County',how='left')# merge the amount of beds per county

In [20]:
df = pd.merge(df,med_hospitals,left_on='County',right_on = 'County',how='left')# merge the amount of hospitals per county

In [21]:
df.columns = df.columns.str.replace('NAME','# hospitals')# change the name of this column to a more descriptive name
df.columns = df.columns.str.replace('BEDS','hospital_beds')# change the name of the beds column to a more descriptive name

We now have a dataframe with covid data, hospitals, # of beds per county that we can input into our predictive model.

In [22]:
health_workers = pd.read_sas('/Users/aryehgelfand/Downloads/AHRF_2018-2019_SAS/ahrf2019.sas7bdat',encoding = "ISO-8859-1") # imported healthcare resurces data

We imported a dataset that has healthcare resources and demographic informations and we can use this to predict whether a county is a likely hotspot for covid-19.

In [23]:
codes = pd.read_csv('../../data/health_resource.csv')# imported codes for understanding data columns

We imported a dataset that has codes that explain each dataset. That way, we can use each column to interpret our health resource data and understand what each column is saying without having to lookup each code.

In [24]:
codes.dropna(inplace=True)
codes.drop(columns=['COL-COL'])# drop col-col column

,FIELD,YEAR OF DATA,VARIABLE NAME
1,F00001,,Blank
2,F00002,,Header - FIPS St & Cty Code
3,F00003,,Entity of File
4,F00004,,Secondary Entity Of File
5,F00005,,Date of File
...,...,...,...
8001,F15338-09,2009,Days w/8-hr Avg Ozone ovr NAAQS
8002,F15338-08,2008,Days w/8-hr Avg Ozone ovr NAAQS
8003,F15338-07,2007,Days w/8-hr Avg Ozone ovr NAAQS
8004,F15338-06,2006,Days w/8-hr Avg Ozone ovr NAAQS


In [25]:
codes['FIELD'] = codes['FIELD'].str.replace('F','f')# conform codes df to healthcare df

In [26]:
codes_dict_keys = {i.strip() for i in codes['FIELD']}# strip white space
codes_dict_values = {i.strip() for i in codes['VARIABLE NAME']}# strip white space

In [27]:
codes_dict = dict(zip(codes_dict_keys,codes_dict_values))# put codes in a dictionary
codes_dict_keys = {key.replace('-',''): key for key in codes_dict.keys()}# take out dash
codes_dict = dict(zip(codes_dict_keys,codes_dict_values))# put codes and meaning into a dict

In [28]:
health_workers.columns = health_workers.columns.map(codes_dict)# convert healthcare codes to their meaning

Now our healthcare resources data has the columns as labels instead of a code that we would have had to lookup each time.

In [29]:
health_workers.columns.values[8] = "State"# rename the eight column with state


In [30]:
health_workers.columns.values[9] = "County"# rename county colum with county

In [31]:
health_workers =  health_workers.loc[:, health_workers.columns.notnull()]# take out columns that are empty

Now we have a huge datframe with over 2k pieces of health data about each county and we have each column correctly labeled.

In [32]:
df = pd.merge(df,health_workers,left_on='County',right_on = 'County',how='left')# merge it onto our original dataframe

In [33]:
df.fillna(0,inplace=True)# fill null with 0

In [35]:
df.drop_duplicates(inplace=True)# drop duplicates from the dataframe

In [36]:
df.columns = df.columns.str.replace("State_x",'State')# make sure that state columns is correectly labeled.

In [37]:
df.drop(df.loc[df['State'] == 'AK'].index, inplace=True)
df.drop(df.loc[df['State'] == 'HI'].index, inplace=True)
df.drop(df.loc[df['State'] == 'PR'].index, inplace=True)

In [38]:
df.drop(columns='County Name',inplace=True)

Now we have a dataframe without counties outside the contigous U.S and without dupicates

In [39]:
df = df.loc[:, ~df.columns.str.contains('Pop')]# taking columns that have the word pop in it.
df = df.loc[:, ~df.columns.str.contains('ins')]# taking columns that have the word ins in it.
df = df.loc[:, ~df.columns.str.contains('Pers')]# taking columns that have the word pers in it.
df = df.loc[:, ~df.columns.str.contains('Persons')]# taking columns that have the word persons in it.
df = df.loc[:, ~df.columns.str.contains('Hsehlds')]
df = df.loc[:, ~df.columns.str.contains('persons')]
df = df.loc[:, ~df.columns.str.contains('Pov')]
df = df.loc[:, ~df.columns.str.contains('families')]
df = df.loc[:, ~df.columns.str.contains('Division')]
df = df.loc[:, ~df.columns.str.contains('Stat')]
df = df.loc[:, ~df.columns.str.contains('HPSA')]
df = df.loc[:, ~df.columns.str.contains('Contiguous')]
df = df.loc[:, ~df.columns.str.contains('Family')]
df = df.loc[:, ~df.columns.str.contains('Lab')]
df = df.loc[:, ~df.columns.str.contains('Housing')]
df = df.loc[:, ~df.columns.str.contains('Salaried')]
df = df.loc[:, ~df.columns.str.contains('Code')]
df = df.loc[:, ~df.columns.str.contains('Workers')]
df = df.loc[:, ~df.columns.str.contains('Financial')]
df = df.loc[:, ~df.columns.str.contains('Unemployed')]
df = df.loc[:, ~df.columns.str.contains('Births')]
df = df.loc[:, ~df.columns.str.contains('Poverty')]
df = df.loc[:, ~df.columns.str.contains('HlthIns')]
df = df.loc[:, ~df.columns.str.contains('Units')]
df = df.loc[:, ~df.columns.str.contains('w/HlthIns')]
df = df.loc[:, ~df.columns.str.contains('Medicaid')]
df = df.loc[:, ~df.columns.str.contains('5-Yr')]
df = df.loc[:, ~df.columns.str.contains('3-Yr')]
df = df.loc[:, ~df.columns.str.contains('Hispanic/Lat')]
df = df.loc[:, ~df.columns.str.contains('Non-Hisp')]
df = df.loc[:, ~df.columns.str.contains('Ins')]
df = df.loc[:, ~df.columns.str.contains('Families')]
df = df.loc[:, ~df.columns.str.contains('Hispanic')]
df = df.loc[:, ~df.columns.str.contains('Asian')]
df = df.loc[:, ~df.columns.str.contains('Households')]
df = df.loc[:, ~df.columns.str.contains('Race')]
df = df.loc[:, ~df.columns.str.contains('Workers')]
df = df.loc[:, ~df.columns.str.contains('Wrkrs')]
df = df.loc[:, ~df.columns.str.contains('Hispanic/Latino')]
df = df.loc[:, ~df.columns.str.contains('White')]
df = df.loc[:, ~df.columns.str.contains('Labor')]
df = df.loc[:, ~df.columns.str.contains('HHld')]
df = df.loc[:, ~df.columns.str.contains('Labor')]
df = df.loc[:, ~df.columns.str.contains('Rce')]
df = df.loc[:, ~df.columns.str.contains('Hsp/Lat')]
df = df.loc[:, ~df.columns.str.contains('Units')]
df = df.loc[:, ~df.columns.str.contains('Age')]
df = df.loc[:, ~df.columns.str.contains('Insurance')]
df = df.loc[:, ~df.columns.str.contains('Parent')]
df = df.loc[:, ~df.columns.str.contains('Income')]
df = df.loc[:, ~df.columns.str.contains('Births')]
df = df.loc[:, ~df.columns.str.contains('Black/Afican')]
df = df.loc[:, ~df.columns.str.contains('American')]
df = df.loc[:, ~df.columns.str.contains('Ratio')]
df = df.loc[:, ~df.columns.str.contains('Air')]
df = df.loc[:, ~df.columns.str.contains('Median')]
df = df.loc[:, ~df.columns.str.contains('Hsp')]
df = df.loc[:, ~df.columns.str.contains('Txc')]
df = df.loc[:, ~df.columns.str.contains('Coll-Hisp/Lat')]
df = df.loc[:, ~df.columns.str.contains('Land')]
df = df.loc[:, ~df.columns.str.contains('Expenses')]
df = df.loc[:, ~df.columns.str.contains('Hhlds')]
df = df.loc[:, ~df.columns.str.contains('Unspecified')]
df = df.loc[:, ~df.columns.str.contains('Yrs')]
df = df.loc[:, ~df.columns.str.contains('Accreditation')]
df = df.loc[:, ~df.columns.str.contains('Administration')]
df = df.loc[:, ~df.columns.str.contains('Inc')]
df = df.loc[:, ~df.columns.str.contains('Percent')]
df = df.loc[:, ~df.columns.str.contains('Employed')]
df = df.loc[:, ~df.columns.str.contains('Number')]
df = df.loc[:, ~df.columns.str.contains('Civilian')]
df = df.loc[:, ~df.columns.str.contains('Enrollees')]
df = df.loc[:, ~df.columns.str.contains('Costs')]
df = df.loc[:, ~df.columns.str.contains('Based')]
df = df.loc[:, ~df.columns.str.contains('Expenses')]
df = df.loc[:, ~df.columns.str.contains('Payroll')]
df = df.loc[:, ~df.columns.str.contains('Area')]
df = df.loc[:, ~df.columns.str.contains('Divorced')]
df = df.loc[:, ~df.columns.str.contains('Administrat')]
df = df.loc[:, ~df.columns.str.contains('Cst')]
df = df.loc[:, ~df.columns.str.contains('Recipients')]
df = df.loc[:, ~df.columns.str.contains('Work')]
df = df.loc[:, ~df.columns.str.contains('Marketplc')]
df = df.loc[:, ~df.columns.str.contains('Expenditure(1000\'\s)')]
df = df.loc[:, ~df.columns.str.contains('Research')]
df = df.loc[:, ~df.columns.str.contains('Teaching')]
df = df.loc[:, ~df.columns.str.contains('Trainees')]
df = df.loc[:, ~df.columns.str.contains('Resident')]
df = df.loc[:, ~df.columns.str.contains('Residents')]
df = df.loc[:, ~df.columns.str.contains('Date')]
df = df.loc[:, ~df.columns.str.contains('Emission')]
df = df.loc[:, ~df.columns.str.contains('Graduates')]
df = df.loc[:, ~df.columns.str.contains('Beneficiaries')]
df = df.loc[:, ~df.columns.str.contains('Unemployment')]
df = df.loc[:, ~df.columns.str.contains('Diploma')]

/Users/aryehgelfand/opt/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


Our over 2k columns have too much info
rmation because we also have census information that we have collected, so we need to remove any columns that relate to demographic information. By, going through the column names, I was able to take out columns that contained certain words and avoid having duplicate information in our dataset. This got our column names down to under 1k. 

In [40]:
health_final = df[['County','total_covid','countyFIPS','cases','deaths','hospital_beds','# hospitals']]

We need to segment our data even further by only keeping columns that have a specific medical resource in them and futher get the amout of columns down from 1k

In [41]:
total = df.loc[:, df.columns.str.contains('Total')]# get columns that have th total
beds = df.loc[:, df.columns.str.contains('Beds')]# get the amount of beds columns
MD = df.loc[:, df.columns.str.contains('M.D')]# get the amount of M.D related columns
hos =  df.loc[:, df.columns.str.contains('Hosp')]
PA = df.loc[:, df.columns.str.contains('Physician Assistants')]
nurses = df.loc[:, df.columns.str.contains('Nurses')]
nurse = df.loc[:, df.columns.str.contains('Nurse')]
DO = df.loc[:, df.columns.str.contains('DO')]
surgery = df.loc[:, df.columns.str.contains('Surgery')]
rt = df.loc[:, df.columns.str.contains('Respiratory Therapists')]
psych = df.loc[:, df.columns.str.contains('Psychiatry')]
nur = df.loc[:, df.columns.str.contains('Nurs')]

Then we need to merge these segmented columns with the final one and get only thes important 600 columns

In [42]:
df1 = health_final.merge(
        total, how="left", left_index=True, right_index=True# merge each of these segmented columns onto each other
    )
df2 = df1.merge(
        beds, how="left", left_index=True, right_index=True
    )
df3 = df2.merge(
        MD, how="left", left_index=True, right_index=True
    )
df4 = df3.merge(
        hos, how="left", left_index=True, right_index=True
    )
df5 = df4.merge(
        PA, how="left", left_index=True, right_index=True
    )
df6 = df5.merge(
        nurses, how="left", left_index=True, right_index=True
    )
df7= df6.merge(
        nurse, how="left", left_index=True, right_index=True
    )
df8= df7.merge(
        DO, how="left", left_index=True, right_index=True
    )
df9= df8.merge(
        surgery, how="left", left_index=True, right_index=True
    )
df10= df9.merge(
        rt, how="left", left_index=True, right_index=True
    )
df11= df10.merge(
        psych, how="left", left_index=True, right_index=True
    )
df13= df11.merge(
        nur, how="left", left_index=True, right_index=True
    )

In [43]:
health_final = df13# make this merged column with only these segmented columns and our original data related to covid

health_final.drop_duplicates(inplace=True)

health_final.fillna(0,inplace=True)

health_final = health_final.loc[:,~health_final.columns.duplicated()]

health_final['County'].dropna(inplace=True)

/Users/aryehgelfand/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:4494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(result)


We end up with a column with all of our desired data only about 640 features. We know that these relate only to specific medical resources or covid related data. 

In [ ]:
#health_final.to_csv('health_resources.csv')